# Not final yet (but soon...)

In [ ]:
# TEMPORARY INSTALL: this will become "pip install langchain[+version specifier]"
!pip install --quiet "git+https://github.com/hemidactylus/langchain@SL-astra-db#egg=langchain&subdirectory=libs/langchain"

! pip install --quiet \
    "astrapy>=0.5.0" \
    "datasets==2.14.6" \
    "openai==0.28.1" \
    "tiktoken==0.5.1"

In [2]:
import os
from getpass import getpass

from datasets import load_dataset

from langchain.vectorstores import AstraDB
from langchain.schema import Document
from langchain.embeddings import OpenAIEmbeddings

## Init

Example values:
- API Endpoint: `https://01234567-89ab-cdef-0123-456789abcdef-us-east1.apps.astra.datastax.com`
- Token: `AstraCS:6gBhNmsk135....` (it must have a role of at least "Database Administrator")
- Keyspace. Optional, if provided: `my_keyspace`
- OpenAI API key: `sk-4fQ3F...`

In [3]:
if 'ASTRA_DB_API_ENDPOINT' not in os.environ:
    os.environ["ASTRA_DB_API_ENDPOINT"] = input("ASTRA_DB_API_ENDPOINT = ")

if 'ASTRA_DB_APPLICATION_TOKEN' not in os.environ:
    os.environ["ASTRA_DB_APPLICATION_TOKEN"] = getpass("ASTRA_DB_APPLICATION_TOKEN = ")

if 'ASTRA_DB_KEYSPACE' not in os.environ:
    ks = input("(Optional) ASTRA_DB_KEYSPACE = ")
    if ks:
        os.environ["ASTRA_DB_KEYSPACE"] = ks

In [4]:
if 'OPENAI_API_KEY' not in os.environ:
    os.environ["OPENAI_API_KEY"] = input("OPENAI_API_KEY = ")

In [6]:
embe = OpenAIEmbeddings()
vstore = AstraDB(
    embedding=embe,
    collection_name="astra_vector_demo",
    token=os.environ["ASTRA_DB_APPLICATION_TOKEN"],
    api_endpoint=os.environ["ASTRA_DB_API_ENDPOINT"],
    namespace=os.environ.get("ASTRA_DB_KEYSPACE"),     # <-- this will disappear in a typical quickstart
)

## Load a small dataset

In [7]:
philo_dataset = load_dataset("datastax/philosopher-quotes")["train"]
print("An example entry:")
print(philo_dataset[16])

An example entry:
{'author': 'aristotle', 'quote': 'Love well, be loved and do something of value.', 'tags': 'love;ethics'}


## Add dataset documents to the vector store

In [8]:
docs = []
for entry in philo_dataset:
    metadata = {"author": entry["author"]}
    if entry["tags"]:
        for tag in entry["tags"].split(";"):
            metadata[tag] = "y"
    doc = Document(page_content=entry["quote"], metadata=metadata)
    docs.append(doc)

inserted_ids = vstore.add_documents(docs)
print(f"\nInserted {len(inserted_ids)} documents.")


Inserted 450 documents.


> TODO: show other add options (add_texts)

## ANN, simple

In [9]:
results = vstore.similarity_search("Our life is what we make of it", k=3)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* We are what we are because we have been what we have been. [{'author': 'freud', 'history': 'y'}]
* We become what we contemplate. [{'author': 'plato', 'knowledge': 'y', 'ethics': 'y'}]
* In the blessings as well as in the ills of life, less depends upon what befalls us than upon the way in which it is met. [{'author': 'schopenhauer', 'knowledge': 'y', 'ethics': 'y'}]


### Similarity values

In [10]:
results = vstore.similarity_search_with_score("Our life is what we make of it", k=3)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.934110] We are what we are because we have been what we have been. [{'author': 'freud', 'history': 'y'}]
* [SIM=0.932188] We become what we contemplate. [{'author': 'plato', 'knowledge': 'y', 'ethics': 'y'}]
* [SIM=0.928739] In the blessings as well as in the ills of life, less depends upon what befalls us than upon the way in which it is met. [{'author': 'schopenhauer', 'knowledge': 'y', 'ethics': 'y'}]


## ANN, with metadata

In [ ]:
results = vstore.similarity_search(
    "Our life is what we make of it",
    k=3,
    filter={"author": "aristotle"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

## MMR search

In [ ]:
results = vstore.max_marginal_relevance_search(
    "Our life is what we make of it",
    k=3,
    filter={"author": "aristotle"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

> TODO: document deletion (various ways)

> TODO: loading from one/more PDFs as well (from url or local file?)

## A mini-RAG

We use the LCEL (langchain expression language), ready for e.g. LangServe and all

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [12]:
retriever = vstore.as_retriever(search_kwargs={'k': 3})

philo_template = """
You are a philosopher that draws inspiration from great thinkers of the past
to craft well-thought answers to user questions. Use the provided context as the basis
for your answers and do not make up new reasoning paths - just mix-and-match what you are given.
Your answers must be concise and to the point, and refrain from answering about other topics than philosophy.

CONTEXT:
{context}

QUESTION: {question}

YOUR ANSWER:"""

philo_prompt = ChatPromptTemplate.from_template(philo_template)

llm = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | philo_prompt 
    | llm 
    | StrOutputParser()
)

In [14]:
chain.invoke("What is the relation between our happiness and the people around us?")

'The relation between our happiness and the people around us is twofold. According to Plato, happiness springs from doing good and helping others, implying that our happiness is intertwined with the well-being of those around us. Kant emphasizes the importance of having someone to love, suggesting that our happiness is influenced by our relationships with others. Therefore, the people around us play a significant role in our happiness.'